# Using FalkorGraphRagCapability with UserProxyAgent for Graph RAG Question Answering

AutoGen provides graph rag integration with Agent Capability. This is an example to integrate FalkorDB (a Knowledge Graph Database).

## Install Falkor DB SDK

In [9]:
%pip install graphrag_sdk==0.1.3b0

2254.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Set OpenAI API

In [ ]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

## Create Knowledge Graph with Your Own Data

Note that, you would need to have a Falkor DB running ready. If you use docker, please set up docker network properly.

In this example, Falker DB endpint is already set at host="172.18.0.2" and port=6379.
For how to set up Falkor DB, please refer to https://docs.falkordb.com/

In [4]:
from graphrag_sdk.schema import Schema

from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine

# Auto generate graph schema from unstructured data
input_path = "../test/agentchat/contrib/graph_rag/the_matrix.txt"

movie_schema = Schema()
actor = movie_schema.add_entity("Actor").add_attribute("name", str, unique=True)
movie = movie_schema.add_entity("Movie").add_attribute("title", str, unique=True)
movie_schema.add_relation("ACTED", actor, movie)

query_engine = FalkorGraphQueryEngine(
    name="IMDB",
    host="172.18.0.2",
    port=6379,
    schema=movie_schema,
)

input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

query_engine.init_db(input_doc=input_documents)

## Create a UserProxyAgent for FalkorDB and Answer Questions

In [8]:
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.graph_rag.falkor_graph_rag_capability import FalkorGraphRagCapability

graph_rag_agent = ConversableAgent(
    name="matrix_agent",
    human_input_mode="NEVER",
)

graph_rag_capability = FalkorGraphRagCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="ALWAYS",
)

user_proxy.initiate_chat(graph_rag_agent, message="Name a few actors who've played in 'The Matrix'")

user_proxy (to matrix_agent):

Name a few actors who've played in 'The Matrix'

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

The actors who've played in 'The Matrix' include:

- Keanu Reeves
- Laurence Fishburne
- Carrie-Anne Moss
- Hugo Weaving

--------------------------------------------------------------------------------
user_proxy (to matrix_agent):

List additional actors.

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

It appears that the actors I initially provided exhaust the list available in our current knowledge graph. There are no additional actors listed for 'The Matrix' beyond Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving. If the film had more actors, they are not present in our graph at this time.

--------------------------------------------------------------------------------
user_proxy (to matrix_agent):

Who is L

ChatResult(chat_id=None, chat_history=[{'content': "Name a few actors who've played in 'The Matrix'", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "The actors who've played in 'The Matrix' include:\n\n- Keanu Reeves\n- Laurence Fishburne\n- Carrie-Anne Moss\n- Hugo Weaving", 'role': 'user', 'name': 'matrix_agent'}, {'content': 'List additional actors.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "It appears that the actors I initially provided exhaust the list available in our current knowledge graph. There are no additional actors listed for 'The Matrix' beyond Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving. If the film had more actors, they are not present in our graph at this time.", 'role': 'user', 'name': 'matrix_agent'}, {'content': 'Who is Lana Wachowski?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "I'm sorry, I don't have an answer for that.", 'role': 'user', 'name': 'matrix_agent'}], summary="I'm sorry, I don't have a